In [2]:
# Run this cell to import all the necessary components
from include.init_rocket.CustomRocket import CustomRocket
%run -i include/widgets/widgets.py

# Rocket construction
## Geometry and Mass 
The first step is to chose the geometry and mass properties of your rocket. Each component (nose, tube, fins) can have its own density and shape. Fill the following form to specify your rocket's properties.

In [3]:
rocket_properties_widget

## Motor picker

This section will allow you to chose the right motor for your rocket based on the desired impulse class and diameter.

$(\textit{Motors data is loaded from the JSON file :  } \texttt{include/widgets/motor\_data.json})$

You will have to chose the two following properties in order to access the motor choice.
### Impulse class : 
The <b>impulse class</b> is a letter code assigned to a range of total impulse with each letter being twice as powerful as the previous.
- A : 1.26–2.50 (N.s),
- B : 2.51–5.00,
- C : 5.01–10.0,
- D : 10.01–20.0,
- E : 20.01–40.0,
- F : 40.01–80.0,
- G : 80.01–160,
- H : 160.01–320,
- I : 320.01–640,
- J : 640.01–1,280,
- K : 1,280.01–2,560,
- L : 2,560.01–5,120,
- M : 5,120.01–10,240,
- N : 10,240.01–20,480,
- O : 20,480.01–40,960
### Motor geometry :
The main diameter of the motor is expressed in millimeters.

The position of the motor corresponds to the distance between the bottom of the motor and the bottom of the tube in meters.

In [4]:
motor_selection_widget

Accordion(children=(Dropdown(description='Impulse Class (N.s):', options=(('[1.26 - 2.50]', 'A'), ('[2.51 - 5.…

## Additional masses
In this section you can create additional mass component to add to your rocket.

These components are represented as cylinders or empty cylinders, but can play the role of various inner components for your rocket such as the parachute or a partition.

In [5]:
additional_masses_widget

Accordion(children=(VBox(children=(BoundedIntText(value=0, description='Number of additional masses', style=De…

## Creation of the rocket
All properties you have previously entered will be stored in the $\texttt{'my\_rocket'}$ dictionary as you run this cell.

You can also create your rocket by directly specifying your rocket's properties in the dict. All properties must be specified.

(Please refer to the documentation for more details about the format of each entry.)

In [5]:
# Run this cell to create the dictionary
my_rocket = {'tube_length':tube_length.value,
             'tube_radius':tube_radius.value,
             'tube_thickness':tube_thickness.value,
             'tube_density':tube_density.value,
             'nose_type':nose_type.value,
             'nose_parameter':nose_parameter.value,
             'nose_length':nose_length.value,
             'nose_radius':nose_radius.value,
             'nose_thickness':nose_thickness.value,
             'nose_density':nose_density.value,
             'fins_Cr':Cr.value,
             'fins_Ct':Ct.value,
             'fins_Xt':Xt.value,
             'fins_s':s.value,
             'fins_thickness':fins_thickness.value,
             'fins_number':fins_number.value,
             'fins_density':fins_density.value,
             'fins_position':fins_position.value,
             'motor':find_motor(selected_motor.value),
             'motor_position':motor_position.value,
             'additional_masses':mass_dict_list()}

In [6]:
%matplotlib widget

rocket = CustomRocket.fromDict(my_rocket)
rocket.show(method = 'go', opened=True, only_ext=False)

In [7]:
volume, mass, cog, inertia = rocket.get_mass_properties()
print("Volume (m^3)                            = {0}".format(volume))
print("Mass   (kg)                             = {0}".format(mass))
print("Position of the center of gravity (COG) = {0}".format(cog))
print("Inertia matrix expressed at the COG     = {0}".format(inertia[0,:]))
print("                                          {0}".format(inertia[1,:]))
print("                                          {0}".format(inertia[2,:]))

Volume (m^3)                            = 0.03827671310286481
Mass   (kg)                             = 233.6649002855638
Position of the center of gravity (COG) = [1.08679616e+00 4.48770426e-11 3.93494681e-13]
Inertia matrix at expressed at the COG  = [ 4.56359854e-01  6.11351001e-08 -1.09704385e-10]
                                          [ 6.11351001e-08  9.78535464e+01 -6.17177922e-10]
                                          [-1.09704385e-10 -6.17177922e-10  9.78535464e+01]


In [8]:
rocket.get_cog()

array([1.08679616e+00, 4.48770426e-11, 3.93494681e-13])